# Fixing Jitter

## Output prone to shaking in **half-squat position**?

Solution:
- Separate the detection threshold for standing and squatting by a certain distance. Specifically, we set two detection thresholds separated by a certain distance.
- Only when the angle is lower than the lower threshold, the status is set as "squatting"; only when the angle is higher than the higher threshold, the status is set as "standing".
- This design can effectively filter out the result fluctuations caused by small-range fluctuations in the measurement value during the binarization process.

![img6-3.png](resource/img6-3.png) 

```python
if hip_angle<125 and knee_angle<105:
    squat = 1
elif hip_angle>145 and knee_angle>125:
    if squat == 1:
        repetition += 1
    squat = 0
```

## Meaningless title also determined the squatting status?

Solution:
Do not let the squat counter count when MoveNet is not confident with the result.

```python
if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None
```

```python
if (hip_angle is not None) and (knee_angle is not None):
    .........
    text = f"Reps: {repetition}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
else:
    text = f"Reps: {repetition}  Knee: ?  Hip: ?"
```

 ## Other possible sources that trigger jittering?

Solution:

- Avoid random interference (such as background, casual movements) through multiple "measurements".
- Collect multiple image samples and only update the squatting status when several consecutive results indicate squatting (standing).


```python
squatting = 0
CONSECUTIVE_THRESHOLD = 3    # << You can change this!
# squatting == 0: last consecutive 3 confident results from images are all standing
# squatting == CONSECUTIVE_THRESHOLD: last consecutive 3 confident results from images are all squatting
def frame_process(frame):
    global repetition, squat, squatting
    ..........
    if hip_angle<125 and knee_angle<105:
        # if MoveNet result is squatting
        #   if current state is standing, squatting += 1
        #   otherwise, reset squatting to CONSECUTIVE_TH
        if squat == 0:
            squatting += 1
        else:
            squatting = CONSECUTIVE_TH
    elif hip_angle>145 and knee_angle>125:
        # if MoveNet result is standing
        #   if current state is squatting, squatting -= 1
        #   otherwise, reset squatting to 0
        if squat == 1:
            squatting -= 1
        else:
            squatting = 0
    if squatting == 0 and squat == 1:
        repetition += 1
        squat = 0
    elif squatting == CONSECUTIVE_TH and squat == 0:
        squat = 1
    ...........
```

## Final Version

In [10]:
%run Squat_common.ipynb

Libraries imported.

The following functions have been imported:
   movenet(input_image)
   _keypoints_and_edges_for_display(keypoints_with_scores, height, width, keypoint_threshold=0.11)
   draw_prediction_on_image(image, keypoints_with_scores, crop_region=None, close_figure=False, output_image_height=None)
The following global variables have been assigned:
   module, model_name, input_size, KEYPOINT_DICT, KEYPOINT_EDGE_INDS_TO_COLOR



In [11]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

def calculate_hip_joint_angle(keypoints_with_scores, keypoint_threshold = 0.2): 
    keypoints_with_scores = keypoints_with_scores[0][0]
    
    # get the position of keypoints from MoveNet output
    a1y, a1x, a1s = keypoints_with_scores[KEYPOINT_DICT["left_shoulder"]]
    a2y, a2x, a2s = keypoints_with_scores[KEYPOINT_DICT["right_shoulder"]]
    b1y, b1x, b1s = keypoints_with_scores[KEYPOINT_DICT["left_hip"]]
    b2y, b2x, b2s = keypoints_with_scores[KEYPOINT_DICT["right_hip"]]
    c1y, c1x, c1s = keypoints_with_scores[KEYPOINT_DICT["left_knee"]]
    c2y, c2x, c2s = keypoints_with_scores[KEYPOINT_DICT["right_knee"]]

    # calculate angle of left and right body respectively
    angle1 = calculate_angle( (a1y, a1x), (b1y, b1x), (c1y, c1x) )
    angle2 = calculate_angle( (a2y, a2x), (b2y, b2x), (c2y, c2x) )

    # if confident score of keypoints are all above threshold, return the midpoint of two angle
    # otherwise, return None
    if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None

def calculate_knee_joint_angle(keypoints_with_scores, keypoint_threshold = 0.2): 
    keypoints_with_scores = keypoints_with_scores[0][0]

    # get the position of keypoints from MoveNet output
    a1y, a1x, a1s = keypoints_with_scores[KEYPOINT_DICT["left_hip"]]
    a2y, a2x, a2s = keypoints_with_scores[KEYPOINT_DICT["right_hip"]]
    b1y, b1x, b1s = keypoints_with_scores[KEYPOINT_DICT["left_knee"]]
    b2y, b2x, b2s = keypoints_with_scores[KEYPOINT_DICT["right_knee"]]
    c1y, c1x, c1s = keypoints_with_scores[KEYPOINT_DICT["left_ankle"]]
    c2y, c2x, c2s = keypoints_with_scores[KEYPOINT_DICT["right_ankle"]]

    # calculate angle of left and right body respectively
    angle1 = calculate_angle( (a1y, a1x), (b1y, b1x), (c1y, c1x) )
    angle2 = calculate_angle( (a2y, a2x), (b2y, b2x), (c2y, c2x) )

    # if confident score of keypoints are all above threshold, return the midpoint of two angle
    # otherwise, return None
    if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None


In [12]:

def putText(frame, text, color = (0, 255, 0)):
    # Define the text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_position = (50, 50)
    text_scale = 0.65
    text_color = color
    text_thickness = 2

    # Add text annotation on the frame
    cv2.putText(frame, text, text_position, font, text_scale, text_color, text_thickness)

repetition = 0
squat = 0
squatting = 0
CONSECUTIVE_TH = 3    # << You can change this!
def frame_process(frame):
    global repetition, squat, squatting
    # calculate and display angle
    keypoints_with_scores = get_keypoints_with_scores_from_image_with_movenet(frame,input_size,input_size)
    hip_angle = calculate_hip_joint_angle(keypoints_with_scores)
    knee_angle = calculate_knee_joint_angle(keypoints_with_scores)


    # check if confidence is qualified, if yes, proceed to judgement
    if hip_angle is not None and knee_angle is not None:
        # if capture continuous CONSECUTIVE_TH angles in squatting, change squat to 1
        # if capture continuous CONSECUTIVE_TH angles in standing, change squat to 0 and repetition += 1
        if hip_angle<125 and knee_angle<105:
            if squat == 0:
                squatting += 1
            else:
                squatting = CONSECUTIVE_TH
        elif hip_angle>145 and knee_angle>125:
            if squat == 1:
                squatting -= 1
            else:
                squatting = 0
        if squatting == 0 and squat == 1:
            repetition += 1
            squat = 0
        elif squatting == CONSECUTIVE_TH and squat == 0:
            squat = 1
        text = f"Reps: {repetition}  Knee: {int(knee_angle)}  Hip: {int(hip_angle)}"
    else:
        text = f"Reps: {repetition}  Knee: ?  Hip: ?"
    if squat:
        text_color = (0,255,0)
    else:
        text_color = (128,192,64)

    # draw visualized prediction from MoveNet on the frame. Attention: Not recommended, VERY LAGGY!
    #frame = draw_prediction_on_image( tf_image_to_model(frame,600,600)[0]/255, keypoints_with_scores)
    
    
    putText(frame, text, text_color)

    return frame


# Run
# ================
if __name__ == "__main__":
    display(stopButton)
    thread = threading.Thread(target=view, args=(stopButton, 0, 1, frame_process))
    thread.start()



ToggleButton(value=False, button_style='danger', description='Stop', icon='square', tooltip='Description')

None